# HW 2: Agent-Based Modeling of Pandemic Spread (Part 1)

In [1]:
# B) iii. Additional Sensitivity Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# parameters
grid_size = 75
num_agents = 100
initial_infected = 5
initial_susceptible = num_agents - initial_infected
time_steps = 200
infection_probability_p = 0.1
recovery_probability_q = 0.05

# Agent states
SUSCEPTIBLE = 0
INFECTED = 1
RECOVERED = 2

def initialize_agents():
    grid = np.zeros((grid_size, grid_size), dtype=int)
    agents = []

    for _ in range(initial_susceptible):
        x, y = np.random.randint(0, grid_size, size=2)
        agents.append([x, y, SUSCEPTIBLE])
        grid[x, y] = SUSCEPTIBLE

    for _ in range(initial_infected):
        x, y = np.random.randint(0, grid_size, size=2)
        agents.append([x, y, INFECTED])
        grid[x, y] = INFECTED

    return grid, agents

def move_agent_with_social_distancing(agent, movement_prob):
    x, y, state = agent
    if np.random.random() < movement_prob:
        direction = np.random.choice(["up", "down", "left", "right", "stay"])
        if direction == "up" and x > 0:
            x -= 1
        elif direction == "down" and x < grid_size - 1:
            x += 1
        elif direction == "left" and y > 0:
            y -= 1
        elif direction == "right" and y < grid_size - 1:
            y += 1
    return [x, y, state]

def update_agents_with_social_distancing(grid, agents, p, q, movement_prob):
    new_agents = []
    for agent in agents:
        x, y, state = agent

        if state == SUSCEPTIBLE:
            if any(neighbor[2] == INFECTED for neighbor in agents if neighbor[:2] == [x, y]):
                if np.random.random() < p:
                    state = INFECTED

        elif state == INFECTED:
            if np.random.random() < q:
                state = RECOVERED

        new_agent = move_agent_with_social_distancing([x, y, state], movement_prob)
        grid[new_agent[0], new_agent[1]] = new_agent[2]
        new_agents.append(new_agent)

    return grid, new_agents

def run_simulation_with_social_distancing(p, q, movement_prob):
    grid, agents = initialize_agents()
    susceptible_counts, infected_counts, recovered_counts = [], [], []

    for _ in range(time_steps):
        grid, agents = update_agents_with_social_distancing(grid, agents, p, q, movement_prob)

        state_counts = np.bincount([agent[2] for agent in agents], minlength=3)
        susceptible_counts.append(state_counts[SUSCEPTIBLE])
        infected_counts.append(state_counts[INFECTED])
        recovered_counts.append(state_counts[RECOVERED])

    return susceptible_counts, infected_counts, recovered_counts

def analyze_simulation(susceptible_counts, infected_counts, recovered_counts):
    peak_infected = max(infected_counts)
    time_to_peak = infected_counts.index(peak_infected)
    final_recovered = recovered_counts[-1]
    stable_recovery_time = len(recovered_counts) - 1

    for t in range(1, len(recovered_counts)):
        if abs(recovered_counts[t] - recovered_counts[t - 1]) < 0.5:
            stable_recovery_time = t
            break

    return {
        "Peak Infected": peak_infected,
        "Time to Peak": time_to_peak,
        "Final Recovered": final_recovered,
        "Time to Stability": stable_recovery_time
    }

# diff social distancing
social_distancing_levels = [1.0, 0.9, 0.7, 0.5, 0.3, 0.1]

social_distancing_results = []
for movement_prob in social_distancing_levels:
    sus_counts, inf_counts, rec_counts = run_simulation_with_social_distancing(
        p=infection_probability_p, q=recovery_probability_q, movement_prob=movement_prob
    )
    metrics = analyze_simulation(sus_counts, inf_counts, rec_counts)
    metrics["Movement Probability"] = movement_prob
    social_distancing_results.append(metrics)

social_distancing_df = pd.DataFrame(social_distancing_results)
social_distancing_df.set_index("Movement Probability", inplace=True)
social_distancing_df


,Peak Infected,Time to Peak,Final Recovered,Time to Stability
Movement Probability,,,,
1.0,5,0,5,2
0.9,5,0,5,1
0.7,5,0,5,2
0.5,4,0,5,1
0.3,5,0,5,1
0.1,5,0,5,1
